# Python Laziness

The ability to compute things lazily is incredibly important to speed things up. Whenever we're using pure functions, and taking advantage of referential transparency, we can rely on laziness (as it doesn't matter _when_ we compute things).

Let's start by defining a simple function `square` that will serve as a snitch, letting us know anytime something was computed. Please help me out here: I want you to imagine that `square` is actually a REALLY expensive computation, that might take minutes to complete.

In [1]:
from operator import pow

In [6]:
def square(x):
    print("Computing square of '{}'".format(x))
    return pow(x, 2)

In [7]:
square(2)

Computing square of '2'


4

In [8]:
square(3)

Computing square of '3'


9

In [9]:
square(9)

Computing square of '9'


81

We'll be comparing `map` to list comprehensions, so we'll define a simple list to use:

In [10]:
a_list = [1, 2, 3, 4]

### List Comprehensions

Let's start with list comprehensions, that by now, they should be intuitive enough:

In [11]:
[square(x) for x in a_list]

Computing square of '1'
Computing square of '2'
Computing square of '3'
Computing square of '4'


[1, 4, 9, 16]

As you can see, a list comprehension returns the square of the values as expected. To do that, it invokes the `square` function with each element, immediately after the list comprehension is executed.

This is **fine**, is probably the expected behavior. But, what if I didn't want to use the result of the list comprehension immediately? What if I only need the first 2 results of that list comprehension? Was it worth computing every value?

That's why we need something lazy.

## Lazy functions: `map` & `filter`

Let's now get square of the elements using a `map`, and see what it returns:

In [12]:
map(square, a_list)

As you can see, the result is not a list, it's a "`<map at 0x7f6e6df7c6d8>`"something. And more importantly, there was no "print" in our screen. That's because `map` is lazy! It hasn't computed a single value yet.

### Iterators and generators

Map returns an iterator. It's a lazy object that will compute "next" elements in a sequence on demand. If you're not using the objects, they won't be computed.

In [13]:
it = map(square, a_list)
it

To get the _next_ element of an iterator, you have to use the `next` function:

In [14]:
next(it)

Computing square of '1'


1

In [15]:
next(it)

Computing square of '2'


4

In [16]:
next(it)

Computing square of '3'


9

But actually, the most common usage is through a regular for loop:

In [17]:
for elem in map(square, a_list):
    print(elem)

Computing square of '1'
1
Computing square of '2'
4
Computing square of '3'
9
Computing square of '4'
16


### Generator comprehensions

There's a way to create lazy (efficient) iterators while also employing the intuitive and declarative syntax of list comprehensions: **generator comprehensions**

They're expressed just like regular list comprehensions, the only change is that instead of using square brackets, you have to use parenthesis:

In [22]:
(square(x) for x in a_list)

<generator object <genexpr> at 0x7f6e6dfc16d0>

This will work in the same way, but lazily:

In [23]:
it = (square(x) for x in a_list)
it

<generator object <genexpr> at 0x7f6e6df5ddb0>

In [24]:
next(it)

Computing square of '1'


1

In [25]:
next(it)

Computing square of '2'


4

In [26]:
next(it)

Computing square of '3'


9

### A good example of the importance of iterators

A good example of the importance of iterator is to avoid duplicated evaluations in list comprehensions. Let's modify our previous list comprehension to return only the squared elements **_that are even_**:

In [21]:
[square(x) for x in a_list if square(x) % 2 == 0]

Computing square of '1'
Computing square of '2'
Computing square of '2'
Computing square of '3'
Computing square of '4'
Computing square of '4'


[4, 16]

As you can see, elements `2` and `4` are computed twice, once to evaluate the `if` expression (the `filter` part) and another to transform the element (the `map` part).

One way to avoid this is by using generator comprehensions, check it out:

In [28]:
squares = (square(x) for x in a_list)

[sq for sq in squares if sq % 2 == 0]

Computing square of '1'
Computing square of '2'
Computing square of '3'
Computing square of '4'


[4, 16]

We could also put everything in the same line:

In [29]:
[sq for sq in (square(x) for x in a_list) if sq % 2 == 0]

Computing square of '1'
Computing square of '2'
Computing square of '3'
Computing square of '4'


[4, 16]